In [9]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from google.colab import output
output.enable_custom_widget_manager()

In [11]:
from google.colab import output
output.disable_custom_widget_manager()

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

Then you need to install Git-LFS. Uncomment the following instructions:

In [12]:
 !apt install git-lfs 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.


Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [13]:
import transformers

print(transformers.__version__)

4.24.0


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/question-answering).

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [14]:
from datasets import load_dataset, load_metric

For our example here, we'll use the [SQUAD dataset](https://rajpurkar.github.io/SQuAD-explorer/). The notebook should work with any question answering dataset provided by the 🤗 Datasets library. If you're using your own dataset defined from a JSON or csv file (see the [Datasets documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) on how to load them), it might need some adjustments in the names of the columns used.

In [17]:
squad_v2 = True
datasets = load_dataset("squad_v2" if squad_v2 else "squad")

  0%|          | 0/2 [00:00<?, ?it/s]

The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [18]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

We can see the training, validation and test sets all have a column for the context, the question and the answers to those questions.

To access an actual element, you need to select a split first, then give an index:

In [19]:
datasets["train"][0]

{'id': '56be85543aeaaa14008c9063',
 'title': 'Beyoncé',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'question': 'When did Beyonce start becoming popular?',
 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}

We can see the answers are indicated by their start position in the text (here at character 515) and their full text, which is a substring of the context as we mentioned above.

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [20]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [21]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,5ad28d6fd7d075001a429a2a,House_music,"Atkins, a former member of Cybotron, released Model 500 ""No UFOs"" in 1985, which became a regional hit, followed by dozens of tracks on Transmat, Metroplex and Fragile. One of the most unusual was ""Strings of Life"" by Derrick May, a darker, more intellectual strain of house. ""Techno-Scratch"" was released by the Knights Of The Turntable in 1984 which had a similar techno sound to Cybotron. The manager of the Factory nightclub and co-owner of the Haçienda, Tony Wilson, also promoted acid house culture on his weekly TV show. The Midlands also embraced the late 1980s house scene with illegal parties and more legal dance clubs such as The Hummingbird.",May was a former member of what music group?,"{'text': [], 'answer_start': []}"
1,5a7e1ad270df9f001a8754b7,Translation,"Arabic translation efforts and techniques are important to Western translation traditions due to centuries of close contacts and exchanges. Especially after the Renaissance, Europeans began more intensive study of Arabic and Persian translations of classical works as well as scientific and philosophical works of Arab and oriental origins. Arabic and, to a lesser degree, Persian became important sources of material and perhaps of techniques for revitalized Western traditions, which in time would overtake the Islamic and oriental traditions.",Why are Arabic translation efforts pointless to Western translation traditions?,"{'text': [], 'answer_start': []}"
2,5710a1c4b654c5140001f9e9,Age_of_Enlightenment,"The word ""public"" implies the highest level of inclusivity – the public sphere by definition should be open to all. However, this sphere was only public to relative degrees. Enlightenment thinkers frequently contrasted their conception of the ""public"" with that of the people: Condorcet contrasted ""opinion"" with populace, Marmontel ""the opinion of men of letters"" with ""the opinion of the multitude,"" and d'Alembert the ""truly enlightened public"" with ""the blind and noisy multitude"". Additionally, most institutions of the public sphere excluded both women and the lower classes. Cross-class influences occurred through noble and lower class participation in areas such as the coffeehouses and the Masonic lodges.",Whixh population groups were excluded from most institutions of the public sphere?,"{'text': ['women and the lower classes'], 'answer_start': [553]}"
3,5ad0161a77cf76001a686950,Kievan_Rus%27,"In the north, the Republic of Novgorod prospered because it controlled trade routes from the River Volga to the Baltic Sea. As Kievan Rus' declined, Novgorod became more independent. A local oligarchy ruled Novgorod; major government decisions were made by a town assembly, which also elected a prince as the city's military leader. In the 12th century, Novgorod acquired its own archbishop Ilya in 1169, a sign of increased importance and political independence, while about 30 years prior to that in 1136 in Novgorod was established a republican form of government - elective monarchy. Since then Novgorod enjoyed a wide degree of autonomy although being closely associated with the Kievan Rus.",What year did Novgorod assign its own archbishop?,"{'text': [], 'answer_start': []}"
4,572689745951b619008f7631,Presbyterianism,"The biggest Presbyterian church is the National Presbyterian Church in Mexico (Iglesia Nacional Presbiteriana de México), which has around 2,500,000 members and associates and 3000 congregations, but there are other small denominations like the Associate Reformed Presbyterian Church in Mexico which was founded in 1875 by the Associate Reformed Church in North America. The Independent Presbyterian Church and the Presbyterian Reformed Church in Mexico, the National Conservative Presbyterian Church in Mexico are existing churches in the Reformed tradition.",When was the Associate Reformed Presbyterian Church in Mexico formed?,"{'text': ['1875'], 'answer_start': [315]}"
5,570d6

In [22]:
show_random_elements(datasets["validation"])

,id,title,context,question,answers
0,5727db85ff5b5019007d96fd,Harvard_University,"Harvard's athletic rivalry with Yale is intense in every sport in which they meet, coming to a climax each fall in the annual football meeting, which dates back to 1875 and is usually called simply ""The Game"". While Harvard's football team is no longer one of the country's best as it often was a century ago during football's early days (it won the Rose Bowl in 1920), both it and Yale have influenced the way the game is played. In 1903, Harvard Stadium introduced a new era into football with the first-ever permanent reinforced concrete stadium of its kind in the country. The stadium's structure actually played a role in the evolution of the college game. Seeking to reduce the alarming number of deaths and serious injuries in the sport, Walter Camp (former captain of the Yale football team), suggested widening the field to open up the game. But the stadium was too narrow to accommodate a wider playing surface. So, other steps had to be taken. Camp would instead support revolutionary new rules for the 1906 season. These included legalizing the forward pass, perhaps the most significant rule change in the sport's history.",In what year did Harvard Stadium become the first ever concrete reinforced stadium in the country?,"{'text': ['1903', '1903', '1903'], 'answer_start': [434, 434, 434]}"
1,5ad0322877cf76001a686dff,Imperialism,"It became a moral justification to lift the world up to French standards by bringing Christianity and French culture. In 1884 the leading exponent of colonialism, Jules Ferry declared France had a civilising mission: ""The higher races have a right over the lower races, they have a duty to civilize the inferior"". Full citizenship rights – ‘’assimilation’’ – were offered, although in reality assimilation was always on the distant horizon. Contrasting from Britain, France sent small numbers of settlers to its colonies, with the only notable exception of Algeria, where French settlers nevertheless always remained a small minority.",The English thought bringing what would uplift other regions?,"{'text': [], 'answer_start': []}"
2,5ad14f5a645df0001a2d1710,European_Union_law,"EU Competition law has its origins in the European Coal and Steel Community (ECSC) agreement between France, Italy, Belgium, the Netherlands, Luxembourg and Germany in 1951 following the second World War. The agreement aimed to prevent Germany from re-establishing dominance in the production of coal and steel as members felt that its dominance had contributed to the outbreak of the war. Article 65 of the agreement banned cartels and article 66 made provisions for concentrations, or mergers, and the abuse of a dominant position by companies. This was the first time that competition law principles were included in a plurilateral regional agreement and established the trans-European model of competition law. In 1957 competition rules were included in the Treaty of Rome, also known as the EC Treaty, which established the European Economic Community (EEC). The Treaty of Rome established the enactment of competition law as one of the main aims of the EEC through the ""institution of a system ensuring that competition in the common market is not distorted"". The two central provisions on EU competition law on companies were established in article 85, which prohibited anti-competitive agreements, subject to some exemptions, and article 86 prohibiting the abuse of dominant position. The treaty also established principles on competition law for member states, with article 90 covering public undertakings, and article 92 making provisions on state aid. Regulations on mergers were not included as member states could not establish consensus on the issue at the time.",Which countries did not agree upon the European Coal and Steel Community agreement?,"{'text': [], 'answer_start': []}"
3,572ffce5a23a5019007fcc15,Rhine,"Around 2.5 million years ago (ending 11,600 years ago) 

In [23]:
#Converting the dataset into array/list of objects/dictionaries according to the format required by OpenAI in Prompt and completion form

import json

example_train = []
for i in range(500):

      example_json_t = { 'prompt': { 'title': datasets["train"][i]['title'],'context':datasets["train"][i]['context'] ,'question':datasets["train"][i]['question']}, 'completion': datasets["train"][i]['answers']['text']}
      example_train .append(example_json_t)

json.dump(example_train , open("example_train.json", 'w' ))

example_test = []
for i in range(500):


      example_json_v = { 'prompt': { 'title': datasets["validation"][i]['title'],'context':datasets["validation"][i]['context'] ,'question':datasets["validation"][i]['question']}, 'completion': datasets["validation"][i]['answers']['text']}
      example_test .append(example_json_v)

json.dump(example_test, open("example_validation.json", 'w' ))

